faiss事前準備:

miniconda_python(3.10版本):

https://docs.conda.io/projects/miniconda/en/latest/miniconda-other-installer-links.html

裝完打開anaconda prompt:

做一個環境```conda create -n VScode-env-3.10.13 python=3.10.13```

做完啟動```conda activate VScode-env-3.10.13```

安裝faiss插件

```conda install -c pytorch faiss-cpu=1.7.4 mkl=2021 blas=1.0=mkl```

```conda install -c pytorch/label/nightly faiss-cpu```

關掉prompt 重新開啟VScode 右上角kernel 選擇another kernel->python env->VScode-env-3.10.13

VScode商城:

python

jupyter

import和拿key

沒裝openai_SDK使用:```pip install openai```

沒裝tabulate使用:```pip install tabulate```

沒裝pandas:```pip install pandas```

沒設定key使用:```setx OPENAI_API_KEY “<yourkey>”```

```echo %OPENAI_API_KEY%```可以看你的KEY

In [45]:
import os
from openai import OpenAI
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

# channel type

In [46]:
channel = 1

# instruction在此輸入

In [47]:
instruction_App = "1.You are InApp chatting bot from Flora customer service. 2.When user ask a question, You must based on history questions and answers and reply based on it. 3.If history questions and answers have no result or not related to current question, you must reply only a special mark:'*'."

instruction_Email = "You are email reply bot from Flora customer service. You must answer based on the history questions and answers from Flora's database. If there is 0 history question and answer, you must reply only a special mark:'*'"

instruction_Comment = "You are Apple Appstore comment reply bot from Flora customer service. You must answer based on the history questions and answers from Flora's database. If there is 0 history question and answer, you must reply only a special mark:'*'"

In [48]:
instruction = ""
if channel == 1 :
    instruction = instruction_App
elif channel == 2 :
    instruction = instruction_Email
elif channel == 3 :
    instruction = instruction_Comment

instruction += "\nYou must follow the SOP: \n"

# The rule that Assistant should follow
SOP = [
    "1. Based on history Q&A to answer.",
    "2. If the customer's problem still can not be solved after you replied, reply only a special mark:'*'.",
]

for i in SOP:
    instruction = instruction + i + "\n"

print(instruction)

先準備faiss

讀embeddings資料

並把dataframe轉為numpy

In [49]:
import pandas as pd
import numpy as np

csv_file = 'data\embeddingsV3_3072.csv'
df = pd.read_csv(csv_file)
data = df.to_numpy()
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
0,-0.030886,-0.020970,0.002210,-0.032358,-0.014908,-0.007581,0.005008,0.012017,-0.034472,0.018240,...,0.001131,0.019792,-0.011917,0.007220,-0.018026,0.003841,0.001450,0.020649,-0.003747,0.021291
1,0.013026,0.013851,-0.009579,-0.059783,0.008046,-0.015229,0.002933,0.004625,-0.002641,0.046384,...,0.017729,0.013825,0.001612,-0.019378,0.011802,-0.009483,-0.006945,0.005122,0.004068,0.009393
2,-0.005118,-0.024766,-0.014762,-0.029704,-0.016605,-0.011532,-0.001856,0.039089,0.005199,0.034809,...,0.013769,0.028311,0.001599,-0.015600,0.024998,-0.024624,-0.000467,-0.005411,0.007058,0.008451
3,-0.019629,0.002125,-0.002155,-0.013240,0.015541,-0.034832,-0.044084,0.013315,0.035132,0.020191,...,0.003723,0.032956,-0.020729,-0.005295,0.016266,0.004657,-0.020029,0.008870,0.011921,0.033432
4,0.005265,0.015462,0.002796,-0.057215,0.013133,-0.032900,-0.001634,0.008408,-0.005457,0.024303,...,0.020498,0.016450,0.004387,-0.020389,-0.004006,0.007847,0.003600,0.005015,0.010267,0.010481


確認內容

In [50]:
print(data.shape[0])
print(type(data))
print(data.dtype)
print(data[0][0])

669
<class 'numpy.ndarray'>
float64
-0.0308859776705503


寫入維度 並把data加入faiss的index

In [51]:
import faiss

index = faiss.IndexFlatL2(data.shape[1])
index.add(data)

創assistant,thread!

In [52]:
from tabulate import tabulate

assistant = client.beta.assistants.create(
    name="Flora's Customer Service",
    instructions=instruction,
    model="gpt-3.5-turbo-1106"
)

thread = client.beta.threads.create()

# input在此輸入
# channel在此輸入(1.App 2.Email 3.comment)

使用同一個Assistant和Thread從這邊開始run (記得把最後刪除Assitant的code註解掉)

In [53]:
input = "Recommend some apps that can help me improve my English proficiency."
prompt_type = 1

做問題的embedding

TOP-k的設定並查找問題

拿答案

In [54]:
def get_similar(input):
    response = client.embeddings.create(input=input, model="text-embedding-3-large", dimensions=3072)
    question = np.array(response.data[0].embedding, ndmin=2)
    k = 3
    distances, indices = index.search(question, k)
    csv_question = 'data\manual.csv'
    qdf = pd.read_csv(csv_question)
    qarray = qdf.to_numpy()
    answer = [(qarray[i], float(dist)) for dist, i in zip(distances[0], indices[0])]
    history = ""
    min_distance = 1.3
    Qcount = 0
    for i in range(0,k):
        if answer[i][1]<min_distance:
            Qcount+=1
            history = history +str(i+1)+". "+ answer[i][0][0]+"\n"+answer[i][0][1]+"\n"
    return Qcount, history

距離設定dist內才顯示

# Prompt內容在此

In [55]:
similar = get_similar(input)
if prompt_type == 1:
    prompt = "Here is "+str(similar[0])+" history questions and answers similar to this question according to Flora's database\n"+similar[1]+"Here is my question:\n"+input
elif prompt_type == 2:
    prompt = input
print(prompt)

Here is 2 history questions and answers related to this question according to Flora's database
1. Can I some apps that I can access during focusing? 
There are two ways to use other apps while growing a tree:  1. If you only need to use other apps occasionally, simply press the 'Pause' button when growing a tree. Then you can go to any other apps without killing the tree. When done, go back to Flora and press the 'Resume' button to continue the session. The pause time will be subtracted from the final duration of the focus session. Note that the Pause feature is available only when there is no price associated with the tree. To see how the Price feature works, please click on the '?' (question mark) icon next to the 'Price' menu on the main screen.  2. If you constantly need certain apps for work, go to the 'More' screen and switch the 'Allow apps during Focus' option on. Then, you can select white-listed apps. During a focus session, you can leave the Flora app without killing the tre

In [56]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content= prompt
)

沒裝tabulate很醜

tabulate文件:https://pypi.org/project/tabulate/

表示assistant,thread,message內容

In [57]:
print(tabulate(assistant,headers=['assistant','content'],tablefmt="simple_outline"))

┌──────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ assistant    │ content                                                                                                                                                                                                                                                                                    │
├──────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ id           │ asst_tWz4KRTyxbh8aRS9bhxLEqiT                                                

In [58]:
print(tabulate(thread,headers=['thread','content'],tablefmt="simple_outline"))

┌────────────┬─────────────────────────────────┐
│ thread     │ content                         │
├────────────┼─────────────────────────────────┤
│ id         │ thread_1H5Z0jaaxqqfEACZXdSQMKHV │
│ created_at │ 1711964420                      │
│ metadata   │ {}                              │
│ object     │ thread                          │
└────────────┴─────────────────────────────────┘


In [59]:
print(tabulate(message,headers=['message','content'],tablefmt="simple_outline"))

┌────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

這次的run

可以使用新instruction複寫assisstant裡的instruction!

like:

```
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)
```


In [60]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

查看run的狀態!

以上如果還沒跑完會顯示queued!

以上如果跑完會顯示completed!

In [61]:
# run = client.beta.threads.runs.retrieve(
#   thread_id=thread.id,
#   run_id=run.id
# )
# print(tabulate(run,headers=['run','content'],tablefmt='simple_outline'))

while run.status == "queued" or run.status == "in_progress":
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
print(run.status)
print(tabulate(run,headers=['run','content'],tablefmt='simple_outline'))

completed
┌─────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ run             │ content                                                                                                                                                                                                                                                                                    │
├─────────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ id              │ run_dyIZKfSsZo5lkUNH6Ir5zX9u                           

In [62]:
# if function calling is required, print user input
try:
    print(run.required_action.submit_tool_outputs.tool_calls[0].function.arguments[21:-2])

except AttributeError:
    print("No function calling")


No function calling


In [63]:
if run.status == "requires_action":
    if run.required_action.submit_tool_outputs.tool_calls[0].function.name == "get_history_question_and_answer":
        print("function calling: get_history_question_and_answer")
        similar = get_similar(run.required_action.submit_tool_outputs.tool_calls[0].function.arguments[21:-2])
        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=[
                {
                    "tool_call_id": run.required_action.submit_tool_outputs.tool_calls[0].id,
                    "output": similar[1]
                }
            ]
        )
else:
    print("No function calling")

No function calling


跑完後打開查看history messages!

In [64]:
import time

time.sleep(5)
messages = client.beta.threads.messages.list(
  thread_id=thread.id,
  order='asc'
)
for i in messages.data:
    print(tabulate(i,tablefmt='simple_outline'))

┌────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

訊息顯示

In [65]:
for i in messages.data:
  print(i.content[0].text.value)
  print("\n")

Here is 2 history questions and answers related to this question according to Flora's database
1. Can I some apps that I can access during focusing? 
There are two ways to use other apps while growing a tree:  1. If you only need to use other apps occasionally, simply press the 'Pause' button when growing a tree. Then you can go to any other apps without killing the tree. When done, go back to Flora and press the 'Resume' button to continue the session. The pause time will be subtracted from the final duration of the focus session. Note that the Pause feature is available only when there is no price associated with the tree. To see how the Price feature works, please click on the '?' (question mark) icon next to the 'Price' menu on the main screen.  2. If you constantly need certain apps for work, go to the 'More' screen and switch the 'Allow apps during Focus' option on. Then, you can select white-listed apps. During a focus session, you can leave the Flora app without killing the tre

# Delete Assistant

In [66]:
response = client.beta.assistants.delete(assistant.id)
time.sleep(5)
print(response)

AssistantDeleted(id='asst_tWz4KRTyxbh8aRS9bhxLEqiT', deleted=True, object='assistant.deleted')
